In [3]:
import os
from dataclasses import dataclass
from pathlib import Path
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_dirs

import zipfile
import gdown
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size

In [9]:
%pwd

'd:\\My DL Workstation\\Krish\\Complete End to End Deep Learning Project\\research'

In [10]:
os.chdir("../")
%pwd

'd:\\My DL Workstation\\Krish\\Complete End to End Deep Learning Project'

We will construct a class that takes data ingested from zip file.

In [11]:
@dataclass(frozen=True) # This will cause the class to be treated as a variable so that we can access the class from other files
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dirs([self.config.artifacts_root]) # Remember, self.config coming from read_yaml is a configbox, so we can call .artifacts_root safely
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_dirs([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [13]:
# Updating component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self) -> str:
        """fetsh data from URL"""
        
        try:
            data_URL = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("/artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from: {data_URL} into file: {zip_download_dir}")
            
            file_ID = data_URL.split('/')[-2]
            prefix_id = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix_id+file_ID, zip_download_dir)
            logger.info(f"Downloaded data from: {data_URL} into file: {zip_download_dir}")
            
        except Exception as e:
            raise e
        
    def extract_zip(self):
        """extracts zip file into the data directory"""
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
                
        

Update the pipeline:

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip()

except Exception as e:
    raise e

[2023-11-16 17:30:28,175: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-16 17:30:28,177: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-16 17:30:28,177: INFO: common: created directory: artifacts]
[2023-11-16 17:30:28,178: INFO: common: created directory: artifacts/data_ingestion]
[2023-11-16 17:30:28,178: INFO: 3937644843: Downloading data from: https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file: artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=4e8e2bf1-b87f-462f-852f-a9c72e8af438
To: d:\My DL Workstation\Krish\Complete End to End Deep Learning Project\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:05<00:00, 10.1MB/s]

[2023-11-16 17:30:35,374: INFO: 3937644843: Downloaded data from: https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file: artifacts/data_ingestion/data.zip]
